In [29]:
import asyncio
from poke_env.player.player import Player
from poke_env.player.random_player import RandomPlayer
import time
import pandas as pd

# pd.options.display.max_rows = 9999
# pd.options.display.max_columns = 9999

# pokeType = pd.read_csv('PTC.csv')

# pokeType.iloc[Row selection, Column selection]

# fightAttack = pokeType.iloc[2, 1]

# normalAttack = pokeType.iloc[1, 1]

# flyingAttack = pokeType.iloc[3, 1]

# poisonAttack = pokeType.iloc[4, 1]

# groundAttack = pokeType.iloc[5, 1]

# rockAttack = pokeType.iloc[6, 1]

# bugAttack = pokeType.iloc[7, 1]

# ghostAttack = pokeType.iloc[8, 1]

# steelAttack = pokeType.iloc[9, 1]

# fireAttack = pokeType.iloc[10, 1]

# waterAttack = pokeType.iloc[11, 1]

# grassAttack = pokeType.iloc[12, 1]

# electricAttack = pokeType.iloc[13, 1]

# psychicAttack = pokeType.iloc[14, 1]

# iceAttack = pokeType.iloc[15, 1]

# dragonAttack = pokeType.iloc[16, 1]

# darkAttack = pokeType.iloc[17, 1]

# fairyAttack = pokeType.iloc[18, 1]

# print(fightAttack)
# print(normalAttack)
# print(flyingAttack)
# print(pokeType)

In [5]:
# Dictionary to map the enumerated integer value of a type to the written word of the type in uppercase
# not currently using anymore but keeping just in case
pokeTypeDict = {
	1 : "BUG",
	2 : "DARK",
	3 : "DRAGON",
	4 : "ELECTRIC",
	5 : "FAIRY",
	6 : "FIGHTING",
	7 : "FIRE",
	8 : "FLYING",
	9 : "GHOST",
	10 : "GRASS",
	11 : "GROUND",
	12 : "ICE",
	13 : "NORMAL",
	14 : "POISON",
	15 : "PSYCHIC",
	16 : "ROCK",
	17 : "STEEL",
	18 : "WATER"
}

In [6]:
import itertools
class BattleNode:
	'''	Holds information about the turn in a battle. 
		These nodes are part of a tree that is built by considering 
		all possible options. This tree is then parsed through to 
		find the best choice for the move to make.'''

	id_iter = itertools.count()

	def __init__(self, battle, player_turn, parentNode):
		''' player_active = player's active pokemon
			opp_active = opponents active pokemon
			player_turn = true if player turn, false if not, None if root node'''
		self.id = next(BattleNode.id_iter)
		self.battle = battle
		self.player_turn = player_turn
		self.parent = parentNode
		self.children = []
		self.move = None
		self.value = None
		self.level = None
	
	def addChild(self, child):
		self.children.append(child)

In [7]:
typeChart = pd.read_csv('TypeTableEdited.csv')
typeChart.rename(columns = {'Unnamed: 0' : 'AttackType'}, inplace = True)
typeChart = typeChart.set_index('AttackType')
# each row is an attacking type
# each column is the defending type

# print(typeChart) #uncomment to see table

In [8]:
def buildTree(node):
	'''	Builds a 2 or 3 level tree of possible moves for the current battle state.
		Returns root node which is the current battle state. '''
	if(node.level != None and node.level >= 3):
		if node.player_turn == False:
			raise Exception("Base node is opponent turn. Base nodes should only be for the player's turn.")
		return node #only want 3 layers, so stop making children
	else:
		if(node.player_turn == None or node.player_turn == False): #if first layer (none) or if parent is opponent turn, now it is player turn
			# Add each available move as a node for the player's turn
			for i in range(0, len(node.battle.available_moves)):
				# Add 1 to layer count
				child = BattleNode(node.battle, True, node)
				if node.level == None:
						child.level = 1
				else: child.level = node.level + 1
				child.move = (node.battle.available_moves[i])
				# go to next layer of branch
				buildTree(child)
				# when it comes back, add to parent
				node.addChild(child)

			# Add node(s) for switch ? or only switch on faint
				# make child
				# call buildTree on child
				# add child to parent
		else:
			# If it is the opponent's turn, then nodes are a few of their possible moves
			if len(node.battle.opponent_active_pokemon.moves) == 0:
				# If we do not know any possible moves yet, just return to the parent and don't make another layer
				return
			else:
				# create a node for each of the known moves
				for move in node.battle.opponent_active_pokemon.moves.values(): 
					child = BattleNode(node.battle, False, node)
					if node.level == None:
						child.level = 1
					else: child.level = node.level + 1
					child.move = move 
					# go to next layer of branch
					buildTree(child)
					# when it comes back, add to parent
					node.addChild(child)

				# Add node(s) for switch ? or only switch on faint
					# make child
					# call buildTree on child
					# add child to parent

		# return node, should be the root when all the recursive calls come back
		return node

In [20]:
def showNodeTree(root):
	print("Root: ", root.id, root.player_turn)
	i = 0
	for child in root.children:
		print("\tPlayer Node ", i, " : ", child.id, child.player_turn, child.move, child.value)
		if len(child.children) > 0:
			j = 0
			for c in child.children:
				print("\t\t Opp Node ", j, " : ", c.id, c.player_turn, c.move, c.value)
				if len(c.children) > 0:
					k = 0
					for kid in c.children:
						print("\t\t\tplayer node ", k, " : ", kid.id, kid.player_turn, kid.move, kid.value)
						k+=1
				j+=1
		i+=1

In [25]:
def minmax(node):
	'''	Goes through the battle tree recursively to evaluate the nodes.
		Then chooses a best move and sends that node back to choose_move.'''

	if len(node.children) < 1:
		# This is the last node of a branch, needs a value assigned
		# Prepare some variables
		player_t1 = node.battle.active_pokemon.type_1
		player_t2 = ""
		player_m_t = node.move.type
		move_basepow = node.move.base_power
		opp_t1 = node.battle.opponent_active_pokemon.type_1
		opp_t2 = ""
		
		# Check if our pokemon has 2nd type
		if node.battle.active_pokemon.type_2 != None:
			player_t2 = node.battle.active_pokemon.type_2
		
		# Check if Opponent pokemon has 2nd type
		if node.battle.opponent_active_pokemon.type_2 != None:
			opp_t2 = node.battle.opponent_active_pokemon.type_2
		
		# First: check for stab
		stab_bonus = 1 #Default value, means no bonus

		if player_t2 != "":
			if player_t1 == player_m_t or player_t2 == player_m_t: #if type1 == move type or type 2 == move type, then stab bonus
				stab_bonus =  1.5
		else:
			if player_t1 == player_m_t: #if type1 == move type, then stab bonus
				stab_bonus =  1.5
				
		# Second: Get multiplier
		# Checks the DataFrame in pokeType for the multiplier for this move
		# get multi for type 1
		t1_multi = typeChart.loc[player_m_t.name, opp_t1.name]
		t2_multi = ""
		if opp_t2 != "" : #two types for opponent
			t2_multi = typeChart.loc[player_m_t.name, opp_t2.name]

		# Third: multiply stab, multis, and base power
		if t2_multi != "" :
			node.value = stab_bonus * t1_multi * t2_multi * move_basepow
		else:
			node.value = stab_bonus * t1_multi * move_basepow

		# testing prints
		# print("\t\tPlayer poke: ", node.battle.active_pokemon)
		# print("\t\tPlayer poke t1: ", player_t1)
		# if player_t2!= "": print("\t\tPlayer poke t2: ", player_t2)
		# print("\t\tPlayer move: ", node.move, " type: ", player_m_t, " bp: ", move_basepow)
		# print("\t\tOpp poke: ", node.battle.opponent_active_pokemon)
		# print("\t\tOpp poke t1: ", opp_t1)
		# if opp_t2!= "": print("\t\tOpp poke t2: ", opp_t2)
		# print("\t\tmulti 1: ", t1_multi)
		# if t2_multi!= "": print("\t\tmulti 2: ", t2_multi)
		# print("\t\tnode value: ", node.value)
		# print()
		# if node.value == 0:
		# 	# print out info
		# 	if move_basepow != 0:
		# 		if t1_multi != 0 and t2_multi !=0:
		# 			if t2_multi != "" :
		# 				print(node.move, "************************t1t2value =" , stab_bonus, t1_multi, t2_multi, move_basepow)
		# 			else:
		# 				print(node.move, "************************t1value =" , stab_bonus, t1_multi, move_basepow)
			# else:
				# print("\t", node.move, "bp = 0")
		return
			
	else:
		# go to end of each branch recursively
		for i in range(0, len(node.children)):
			minmax(node.children[i])
		# Comes back when all children have a value
		# Get value for node from children based on turn
		if node.player_turn: 
			minVal = 9999 #picks min of opponent moves
			for child in node.children:
				if child.value < minVal:
					minVal = child.value
			node.value = minVal 
			
		else: #if opponent turn
			maxVal = -9999 #picks max of player moves
			for child in node.children:
				if child.value > maxVal:
					maxVal = child.value
			node.value = maxVal 
	
	# if the node is the root (no parent)
	if node.parent == None:
		# find best child and return their move
		if node.player_turn == None: # root should always be player turn
			# Pick the highest of player moves for chosen move for root
			best_move = None
			maxVal = -9999
			for child in node.children:
				if child.value > maxVal:
					maxVal = child.value
					best_move = child.move
			# print("Chosen move and value: ", best_move, maxVal)
			# if maxVal == 0:
				# print("\t\t\t\t\t\tbp: ", best_move.base_power, "pp: ", best_move.current_pp)
				# print the other possible moves
				# for c in node.children:
				# 	print(c.move, c.value)
				# showNodeTree(node)
			return best_move
		else: 
			print("Error: root not player turn")
			return 

In [24]:
class MinMaxPlayer(Player):
	
	def choose_move(self, battle):
		if battle.available_moves:
			# best_move = battle.available_moves[0] #used to test that the connection to the server is working
			# print("opps moves: \n", battle.opponent_active_pokemon.moves)
			# send in battle object so all properties are available
			rootNode = BattleNode(battle, None, None)
			rootNode = buildTree(rootNode) #send root, returns root node of constructed tree
			
			best_move = minmax(rootNode) # send in root node(current battle), get chosen move
			# showNodeTree(rootNode)
			return self.create_order(best_move)
		else: 
			return self.choose_random_move(battle)

In [33]:
from maxDamage import MaxDamagePlayer

#Create random Opponent
random_player = RandomPlayer(battle_format="gen8randombattle")

#Create maxDamage opponent
maxDamage_player = MaxDamagePlayer(battle_format="gen8randombattle")

#Create minMaxPlayer
mmP = MinMaxPlayer(battle_format="gen8randombattle")

numBattles = 10

start = time.time()
# run for set number of games
await mmP.battle_against(random_player, n_battles=numBattles)

# print results 
print(
	"MinMax player won %d / %d battles against random player [this took %f seconds]"
	% (
		mmP.n_won_battles, numBattles, time.time() - start
	)
)

mmP2 = MinMaxPlayer(battle_format="gen8randombattle")
start = time.time()
await mmP2.battle_against(maxDamage_player, n_battles=numBattles)
# print results 
print(
	"MinMax player won %d / %d battles against maxDamage player [this took %f seconds]"
	% (
		mmP2.n_won_battles, numBattles, time.time() - start
	)
)

MinMax player won 9 / 10 battles against random player [this took 4.230987 seconds]
MinMax player won 4 / 10 battles against maxDamage player [this took 3.962321 seconds]
